In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Decision Trees

In [14]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2)

In [15]:
from sklearn.model_selection import train_test_split
X = pd.read_csv('/home/algoritmia/Dropbox/ml_intro/vol/intermediate_results/intermediate_results/X_opening.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)

In [16]:
X_train, X_test,y_train,y_test = train_test_split(X,y,random_state=1)

In [9]:
!pip install graphviz

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
import graphviz

In [18]:
from sklearn.tree import export_graphviz

treedot = export_graphviz(model,out_file=None, feature_names=X.columns)

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [19]:
graphviz.Source(treedot)

NameError: name 'treedot' is not defined

# Random Forest y Gradient Boosting Trees

### Ensembles

Random Forest y Gradient Bossted Trees, forman parte de una familia de algoritmos que se denominan ensembles.

Ensemble = Submodelos -->Entrenamiento --> Predicciones Intermedias --> Voto -->Prediccion Final

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

forest = RandomForestRegressor(200)
results = cross_validate(forest,X,y,cv=5,scoring='r2')

In [22]:
test_scores = results['test_score']
train_scores = results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

0.9656410291740236
0.5199528367446419


Mejor resultado que Lasso! Ya no tenemos Baias y tenemos mejor score r2. SIn embargo tenemos una diferencia importante entre score de entranamiento y de test (over)

# Gradient Boosted Trees

In [24]:
from sklearn.ensemble import  GradientBoostingRegressor
ensemble = GradientBoostingRegressor()
results = cross_validate(ensemble,X,y,cv=5,scoring='r2')

In [25]:
test_scores = results['test_score']
train_scores = results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

0.9151392143549633
0.52694505508475


#
Optimizacion de hiperparametros

### Grid Search

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)


In [43]:
from sklearn.model_selection import GridSearchCV

param_test1 = {'n_estimators':range(20,501,20)}
estimator = GradientBoostingRegressor(learning_rate=0.1, 
                                       min_samples_split=500,
                                       min_samples_leaf=50,
                                       max_depth=8,
                                       max_features='sqrt',
                                       subsample=0.8,
                                       random_state=10)
gsearch1 = GridSearchCV(estimator, 
                        param_grid = param_test1, 
                        scoring='r2', 
                        cv=5)

In [44]:
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_sa...       subsample=0.8, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(20, 501, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [45]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [46]:
gsearch1.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=500,
             min_weight_fraction_leaf=0.0, n_estimators=240,
             n_iter_no_change=None, presort='auto', random_state=10,
             subsample=0.8, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [47]:
cross_validate(gsearch1.best_estimator_,X_train,y_train)

{'fit_time': array([0.4532423 , 0.2274332 , 0.18496132]),
 'score_time': array([0.05150795, 0.00352383, 0.00346327]),
 'test_score': array([0.64008566, 0.81626875, 0.76780843]),
 'train_score': array([0.86953232, 0.77262356, 0.81279493])}

In [48]:
final_results = cross_validate(gsearch1.best_estimator_,X_train,y_train)

In [49]:
test_scores = final_results['test_score']
train_scores = final_results['train_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

0.8183169366503803
0.74138761251573
